<a href="https://colab.research.google.com/github/Igprad01/Sentimen-Analisis/blob/main/Analisis_Sentimen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ANALISIS SENTIMEN CASE STUDY OPINI PUBLIK TWITTER

In [17]:
#install library
!pip install Sastrawi
!pip install nltk
!pip install -U scikit-learn

In [19]:
#import library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk as nltk
import csv
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

INPUT DATA SET

In [20]:
# load data
files = '/content/sample_data/Gabungan sample crawling JOT jkt48.csv'
df = pd.read_csv(files)
df.head()
# display(df)

,conversation_id_str,created_at,favorite_count,full_text,id_str,image_url,in_reply_to_screen_name,lang,location,quote_count,reply_count,retweet_count,tweet_url,user_id_str,username
0,1.873810e+18,Mon Dec 30 18:55:01 +0000 2024,0,Om JOT Liat deh Si up girls ngajak @RFritzy_JK...,1.873810e+18,NaN,NaN,in,NaN,0,0,0,https://x.com/XJulianto/status/187380500346035...,1.818490e+18,XJulianto
1,1.873640e+18,Mon Dec 30 17:47:27 +0000 2024,0,@RFritzy_JKT48 Kalo oshinya JOT masih ga apa a...,1.873790e+18,https://pbs.twimg.com/media/GgEIPq1bYAExxgW.jpg,RFritzy_JKT48,in,"DKI Jakarta, Indonesia",0,0,0,https://x.com/Anggithaadr/status/1873788000431...,8.198810e+17,Anggithaadr
2,1.873760e+18,Mon Dec 30 15:52:30 +0000 2024,0,harapan untuk jkt48 yaitu membuat para jot sad...,1.873760e+18,NaN,NaN,in,NaN,0,0,0,https://x.com/blq_abilzz/status/18737590745775...,1.786020e+18,blq_abilzz
3,1.873630e+18,Mon Dec 30 15:25:31 +0000 2024,1,@_rybh @Maffi_f07 @Auwia_JKT48 Itu urusan inte...,1.873750e+18,NaN,_rybh,in,Salakanagara,0,1,0,https://x.com/gagitugie___/status/187375228321...,2.429397e+09,gagitugie___
4,1.873630e+18,Mon Dec 30 15:07:03 +0000 2024,0,@gagitugie___ @Maffi_f07 @Auwia_JKT48 Menunjuk...,1.873750e+18,NaN,gagitugie___,in,NaN,0,1,0,https://x.com/_rybh/status/1873747635750011333,3.181792e+09,_rybh


In [25]:
df_text = df['full_text']
df_text.head()

,full_text
0,Om JOT Liat deh Si up girls ngajak @RFritzy_JK...
1,@RFritzy_JKT48 Kalo oshinya JOT masih ga apa a...
2,harapan untuk jkt48 yaitu membuat para jot sad...
3,@_rybh @Maffi_f07 @Auwia_JKT48 Itu urusan inte...
4,@gagitugie___ @Maffi_f07 @Auwia_JKT48 Menunjuk...


PRE-PROSESSING DATA

In [26]:
# case folding
lowercase = df_text.str.lower()
display(lowercase)

,full_text
0,om jot liat deh si up girls ngajak @rfritzy_jk...
1,@rfritzy_jkt48 kalo oshinya jot masih ga apa a...
2,harapan untuk jkt48 yaitu membuat para jot sad...
3,@_rybh @maffi_f07 @auwia_jkt48 itu urusan inte...
4,@gagitugie___ @maffi_f07 @auwia_jkt48 menunjuk...
...,...
6211,lahkk makasih om jot see u @greesel_jkt48 http...
6212,@jambubiji08 @gracie_jkt48 harusnya faham jot om
6213,@greseella @greesel_jkt48 udah kena bedak jot ...
6214,@naim_pasha @officialjkt48 @y_chikajkt48 @f_fe...


In [27]:
#cleaning text
def clean_text(text):
    text = re.sub('http\S+', '', text)
    text = re.sub( 'www\s', '', text)
    text = re.sub('@', '', text)
    text = re.sub('\d+', '', text)
    text = re.sub('\s+', ' ', text)
    text = text.strip()
    return text

df_text = df_text.apply(clean_text)
display(df_text)

,full_text
0,Om JOT Liat deh Si up girls ngajak RFritzy_JKT...
1,RFritzy_JKT Kalo oshinya JOT masih ga apa apa ...
2,harapan untuk jkt yaitu membuat para jot sadar...
3,_rybh Maffi_f Auwia_JKT Itu urusan internal jo...
4,gagitugie___ Maffi_f Auwia_JKT Menunjukan kalo...
...,...
6211,Lahkk makasih om jot See u Greesel_JKT
6212,JambuBiji Gracie_JKT Harusnya faham JOT om
6213,greseella Greesel_JKT Udah kena bedak JOT aja nih
6214,naim_pasha officialJKT Y_ChikaJKT F_FeniJKT st...


In [28]:
#tokenization

In [ ]:
#stopwords removal

In [ ]:
#stemming

PELABELAN DATA

In [ ]:
# pelabelan data


In [ ]:
# modelisasi

In [ ]:
# extrasi fitur

In [ ]:
# klasifikasi sentimen

In [ ]:
# evaluasi hasil


In [ ]:
# visualisasi dan hasilnya